In [ ]:
from pysal.lib import weights
from pysal.explore import esda
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn
import contextily

# 1. Data Setup

In [ ]:
# Reading in the data
path = r'YOUR FILE PATH'

# REMINDER - The listings data must be downloaded from Inside Airbnb
listings = pd.read_csv(path + r'NY Airbnb June 2020\listings.csv.gz', compression='gzip', low_memory=False)

# Converting it to a GeoPandas DataFrame
listings_gpdf = gpd.GeoDataFrame(
    listings,
    geometry=gpd.points_from_xy(listings['longitude'],
                                   listings['latitude'],
                                   crs="EPSG:4326")
)

# Focusing on attractions in Manhattan, so we need to create a mask to filter locations 
# in the Manhattan borough
boroughs = gpd.read_file(path + r"NYC Boroughs\nybb_22a\nybb.shp")
manhattan = boroughs[boroughs['BoroName']=='Manhattan']
manhattan = manhattan.to_crs('EPSG:4326')

listings_mask = listings_gpdf.within(manhattan.loc[3, 'geometry'])

listings_manhattan = listings_gpdf.loc[listings_mask]

In [ ]:
print(list(listings_manhattan.columns))

In [ ]:
# Subsetting the data to a handful of variables that could be indicative of nightly airbnb price
voi = ['id' # Unique identifier for the listing
       ,'room_type' # Type of room
       ,'accommodates' # The maximum capacity of the listing 
       ,'bedrooms' # The number of bedrooms
       ,'beds' # The number of beds
       ,'review_scores_rating' # The rating
       ,'price' # The nightly rental rate, dependent variable (Y)
      ]

listings_manhattan_subset = listings_manhattan[voi]

## 1.1 Exploratory Data Analysis & Processing

In [ ]:
listings_manhattan_subset.dtypes

In [ ]:
for col in listings_manhattan[['room_type']]:
  print(listings_manhattan_subset[col].unique())

In [ ]:
# Encoding categorical variables

# Room Type, All 0s = hotel room
listings_manhattan_subset['rt_entire_home_apartment'] = np.where(listings_manhattan_subset["room_type"]=='Entire home/apt', 1, 0)
listings_manhattan_subset['rt_private_room'] = np.where(listings_manhattan_subset["room_type"]=='Private room', 1, 0)
listings_manhattan_subset['rt_shared_room'] = np.where(listings_manhattan_subset["room_type"]=='Shared room', 1, 0)

In [ ]:
listings_manhattan_subset['price'].values

In [ ]:
# Cleaning up the price column
listings_manhattan_subset['price'] = listings_manhattan_subset['price'].str.replace('$', '')
listings_manhattan_subset['price'] = listings_manhattan_subset['price'].str.replace(',', '')
listings_manhattan_subset['price'] = listings_manhattan_subset['price'].astype(float)

# Logging the price variable
listings_manhattan_subset['log_price'] = np.log(listings_manhattan_subset['price'])

In [ ]:
# Checking for missingness
print('Total Records:', len(listings_manhattan_subset))
listings_manhattan_subset.isna().sum()

In [ ]:
listings_manhattan_subset = listings_manhattan_subset[listings_manhattan_subset['bedrooms'].notna()]
listings_manhattan_subset = listings_manhattan_subset[listings_manhattan_subset['review_scores_rating'].notna()]
listings_manhattan_subset = listings_manhattan_subset[listings_manhattan_subset['beds'].notna()]
listings_manhattan_subset.isna().sum()

# 2. OLS Regression Analysis

In [ ]:
from pysal.model import spreg

In [ ]:
# Defining a list of explanatory variables
m_vars = ['accommodates', 'bedrooms','beds','review_scores_rating',
          'rt_entire_home_apartment','rt_private_room','rt_shared_room'
         ]

In [ ]:
ols_m = spreg.OLS(
    listings_manhattan_subset[["log_price"]].values # the dependent variable (Y)
    ,listings_manhattan_subset[m_vars].values # the independent variables(Xs)
    ,name_y = 'price',
    name_x = m_vars
)

print(ols_m.summary)

## 2.1 Exploring unmodeled relationships

In [ ]:
# Creating a column to store the model residuals
listings_manhattan_subset["ols_m_r"] = ols_m.u

# Bringing back in the neighborhood variable
listings_manhattan_subset = listings_manhattan_subset.merge(listings_manhattan[['id','neighbourhood_cleansed']], how='left',on='id')

# Get average value of the residual by neighborhood
mean = (
    listings_manhattan_subset.groupby("neighbourhood_cleansed")
    .ols_m_r.mean()
    .to_frame("neighborhood_residual")
)

# Creating a dataframe to store the residuals by neighborhood
residuals_neighborhood = listings_manhattan_subset.merge(
        mean, how="left", left_on="neighbourhood_cleansed", right_index=True
    ).sort_values("neighborhood_residual")

In [ ]:
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML

# Plotting the distribution of the residuals in a violin plot
fig = px.violin(residuals_neighborhood, x="neighbourhood_cleansed", y="ols_m_r", color="neighbourhood_cleansed", box=True)
# Updating the x and y axis labels
fig.update_layout(
    xaxis_title="Neighborhood", yaxis_title="Residuals"
)
HTML(fig.to_html())

In [ ]:
import textwrap

In [ ]:
# Producing a map of the average residual by neighborhood
import requests
import geopandas as gpd
from shapely.geometry import shape

# Reading in the NYC Neighborhoods shapefile from data file
import os

nyc_n = gpd.read_file(os.path.join(path,r"NYC Neighborhoods\NYC_Neighborhoods.geojson"), driver='GeoJSON')

# Merging
nyc_n_r = nyc_n.merge(mean, left_on='neighborhood', right_on="neighbourhood_cleansed")

f, ax = plt.subplots(1, figsize=(15, 25))
   
# Plot choropleth of local statistics
nyc_n_r.plot(
    column='neighborhood_residual', 
    cmap='vlag', 
    scheme='quantiles',
    k=4, 
    edgecolor='white', 
    linewidth=0.1, 
    alpha=0.75,
    legend=True,
    ax=ax
)

# Add basemap
contextily.add_basemap(
    ax,
    crs=nyc_n.crs,
    source=contextily.providers.Stamen.Watercolor,
)

for idx, row in nyc_n_r.iterrows():
    txt=row['neighborhood']
    n_txt = '\n'.join(textwrap.wrap(txt, width=11, replace_whitespace=True))
    #print(n_txt)                  
    plt.annotate(text=n_txt, xy=tuple([row.geometry.centroid.x, row.geometry.centroid.y]),
                 horizontalalignment='center',fontsize=14, rotation=50, wrap=True)

    
# Remove axis
ax.set_axis_off()

# Display
plt.show()
#f.savefig(r"G:\My Drive\Geospatial Data Science with Python\Neighborhood Residuals.png")

In [ ]:
# Bringing in the geometry attribute
residuals_neighborhood = residuals_neighborhood.merge(listings_manhattan[['id','geometry']], how='left',on='id')

# Building the spatial weights matrix
knn = weights.KNN.from_dataframe(residuals_neighborhood, k=5)

# Constructing the spatial lag
lag_residual = weights.spatial_lag.lag_spatial(knn, ols_m.u)

# Plotting the results
fig = px.scatter(x=ols_m.u.flatten(), y=lag_residual.flatten(), trendline="ols",
                width=800, height=800)

# Updating the x and y axis labels
fig.update_layout(
    xaxis_title="Airbnb Residuals", yaxis_title="Spatially Lagged Residuals"
)

fig.show()


# 3. OLS Regression Analysis w/ Spatial Atributes

## 3.1 Spatial Feature Engineering

In [ ]:
# Reading in data on popular NYC Attractions
nyc_attr = pd.read_csv(path + 'NYC Attractions\\NYC Attractions.csv')

# Convert PDF to GPDF
nyc_attr_gpdf =  gpd.GeoDataFrame(
    nyc_attr,
    geometry=gpd.points_from_xy(nyc_attr['Longitude'],
                                   nyc_attr['Latitude'],
                                   crs="EPSG:4326")
)

# Calculate the distance to each attraction per airbnb
attractions = nyc_attr_gpdf.Attraction.unique()

# Converting to a projected coordinate system
nyc_attr_gpdf_p = nyc_attr_gpdf.to_crs('EPSG:2263')
listings_manhattan_p = listings_manhattan.to_crs('EPSG:2263')

# Applying a lambda function that calls geopandas distance function to calcuate the distance between each airbnb and each attraction
distances = listings_manhattan_p.geometry.apply(lambda g: nyc_attr_gpdf_p.distance(g))

# Renaming the columns based on the attraction for which the distance is calculated
distances.columns = attractions

# Convert from 'US survey foot' to miles
distances = distances.apply(lambda x: x/5280, axis=1)

# Check to see which locations are less than 1 to 6 miles
distances_1mi = distances.apply(lambda x: x <=1, axis=1).sum(axis=1)
distances_2mi = distances.apply(lambda x: x <=2, axis=1).sum(axis=1)
distances_3mi = distances.apply(lambda x: x <=3, axis=1).sum(axis=1)
distances_4mi = distances.apply(lambda x: x <=4, axis=1).sum(axis=1)
distances_5mi = distances.apply(lambda x: x <=5, axis=1).sum(axis=1)
distances_6mi = distances.apply(lambda x: x <=6, axis=1).sum(axis=1)

# Creating a dataframe combining all the distance bands
distance_df = pd.concat([distances_1mi,distances_2mi,distances_3mi,distances_4mi,distances_5mi,distances_6mi], axis=1)
distance_df.columns = ['Attr_1mi','Attr_2mi','Attr_3mi','Attr_4mi','Attr_5mi','Attr_6mi']

# Joining back to the listings geopandas df
listings_manhattan = listings_manhattan.merge(distances, left_index=True, right_index=True)
listings_manhattan = listings_manhattan.merge(distance_df, left_index=True, right_index=True)

## 3.2 Modeling with Proximity Features

In [ ]:
# Geographic Features
g_vars = ['Central Park', 'Central Park Zoo', 'Empire State Building', 'Statue of Liberty', 
          'Rockeffeller Center', 'Chrysler Building', 'Times Square', 'MoMa', 'Charging Bull']

# Bring the geographic features into our subset
listings_manhattan_subset = listings_manhattan_subset.merge(listings_manhattan[['id','Central Park', 'Central Park Zoo', 'Empire State Building', 'Statue of Liberty', 
          'Rockeffeller Center', 'Chrysler Building', 'Times Square', 'MoMa', 'Charging Bull']], how='left',on='id')

# Adding geographic features to model variables used previously
g_m_vars = m_vars + g_vars

In [ ]:
ols_m_g = spreg.OLS(
    listings_manhattan_subset[["log_price"]].values,
    listings_manhattan_subset[g_m_vars].values,
    name_y = 'price',
    name_x = g_m_vars
)

print(ols_m_g.summary)

In [ ]:
model_diagnostics = pd.DataFrame(
    [[ols_m.r2, ols_m.ar2], [ols_m_g.r2, ols_m_g.ar2]],
    index=["OLS_M", "OLS_M_G"],
    columns=["R2", "Adj. R2"],
)

model_diagnostics

In [ ]:
# Constructing the spatial lag
lag_residual_2 = weights.spatial_lag.lag_spatial(knn, ols_m_g.u)

# Plotting the results
fig = px.scatter(x=ols_m_g.u.flatten(), y=lag_residual_2.flatten(), trendline="ols",
                width=800, height=800, title="Residual Plot for OLS Model with Geospatial Features")

# Updating the x and y axis labels
fig.update_layout(
    xaxis_title="Airbnb Residuals", yaxis_title="Spatially Lagged Residuals"
)

fig.show()

## 3.3 Spatial Fixed Effects Model

In [ ]:
# Bringing back in the neighbourhood_cleansed variable
listings_manhattan_subset = listings_manhattan_subset.merge(listings_manhattan[['id','neighbourhood_cleansed']], how='left',on='id')

# spatial fixed effect model implementation
sfe_m = spreg.OLS_Regimes(
    #The dependent variable (Y) - Log Price (log_price)
    listings_manhattan_subset[["log_price"]].values,
    
    # The independent variables (Xs)
    listings_manhattan_subset[g_m_vars].values,
    
    # Variable which specifies which neighborhood each airbnb falls within
    listings_manhattan_subset["neighbourhood_cleansed"].tolist(),
    
    # Vary constant by each cross-section/group
    constant_regi="many",
    
    # Here we tell the model that the variables are kept constant by group
    cols2regi=[False] * len(g_m_vars),
    
    # Here we tell the model to estimate a single sigma coefficient
    regime_err_sep=False,
    
    # Dependent variable name
    name_y="log_price",
    
    # Independent variables names
    name_x=g_m_vars,
)
# printing the model summary
print(sfe_m.summary)

In [ ]:
# Print Chow Statistic 
sfe_m.chow.joint

In [ ]:
# Constructing the spatial lag
lag_residual_3 = weights.spatial_lag.lag_spatial(knn, sfe_m.u)

# Plotting the results
fig = px.scatter(x=sfe_m.u.flatten(), y=lag_residual_3.flatten(), trendline="ols",
                width=800, height=800, title="Residual Plot for SFE OLS Model")

# Updating the x and y axis labels
fig.update_layout(
    xaxis_title="Airbnb Residuals", yaxis_title="Spatially Lagged Residuals"
)

fig.show()

## 4.0 Geographically Weighted Regression

In [ ]:
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW

In [ ]:
# Encoding categorical variables

# Room Type, All 0s = hotel room
listings_manhattan['rt_entire_home_apartment'] = np.where(listings_manhattan["room_type"]=='Entire home/apt', 1, 0)
listings_manhattan['rt_private_room'] = np.where(listings_manhattan["room_type"]=='Private room', 1, 0)
listings_manhattan['rt_shared_room'] = np.where(listings_manhattan["room_type"]=='Shared room', 1, 0)

# Cleaning up the price column
listings_manhattan['price'] = listings_manhattan['price'].str.replace('$', '')
listings_manhattan['price'] = listings_manhattan['price'].str.replace(',', '')
listings_manhattan['price'] = listings_manhattan['price'].astype(float)

# Logging the price variable
listings_manhattan['log_price'] = np.log(listings_manhattan['price'])

listings_manhattan = listings_manhattan[listings_manhattan['bedrooms'].notna()]
listings_manhattan = listings_manhattan[listings_manhattan['review_scores_rating'].notna()]
listings_manhattan = listings_manhattan[listings_manhattan['beds'].notna()]
listings_manhattan.isna().sum()

listings_manhattan = listings_manhattan[listings_manhattan['bedrooms'].notna()]
listings_manhattan = listings_manhattan[listings_manhattan['review_scores_rating'].notna()]
listings_manhattan = listings_manhattan[listings_manhattan['beds'].notna()]

In [ ]:
# Setting the explanatory variables
exp_vars = listings_manhattan[['accommodates', 'bedrooms','beds','review_scores_rating']].values

# Setting the dependent variable: log_price
y = (listings_manhattan['log_price'].values).reshape((-1,1))

# Defining the coordinates of the observations
coords = list(zip(listings_manhattan.geometry.x,listings_manhattan.geometry.y))

In [ ]:
# Selecting the optimal bandwidth for the explanatory variables
gwr_selector = Sel_BW(coords, y, exp_vars, spherical=True)
gwr_bw = gwr_selector.search(bw_min=2)

# Displaying the optimal bandwidth
gwr_bw

In [ ]:
# Fit the GWR model to your data
gwr_results = GWR(coords, listings_manhattan[["log_price"]].values, 
                     exp_vars, gwr_bw).fit()

# Print the results of the GWR model
print(gwr_results)

In [ ]:
gwr_results.summary()

In [ ]:
residuals = gwr_results.resid_response

In [ ]:
# Constructing the spatial lag
lag_residual_4 = weights.spatial_lag.lag_spatial(knn, residuals)

# Plotting the results
fig = px.scatter(x=residuals.flatten(), y=lag_residual_4.flatten(), trendline="ols",
                width=800, height=800, title="Residual Plot for GWR Model")

# Updating the x and y axis labels
fig.update_layout(
    xaxis_title="Airbnb Residuals", yaxis_title="MGWR Residuals"
)

fig.show()

## 4.1 Multi-scale Geographically Weighted Regression

In [ ]:
from mgwr.gwr import MGWR

In [ ]:
listings_manhattan_sample = listings_manhattan.copy()

In [ ]:
exp_vars = listings_manhattan_sample[['accommodates', 'bedrooms','review_scores_rating']].values
y = (listings_manhattan_sample['price'].values).reshape((-1,1))
coords = list(zip(listings_manhattan_sample.geometry.x,listings_manhattan_sample.geometry.y))

In [ ]:
# Standardizing
exp_vars = (exp_vars - exp_vars.mean(axis=0)) / exp_vars.std(axis=0)
y = (y - y.mean(axis=0)) / y.std(axis=0)

In [ ]:
selector = Sel_BW(coords, y, exp_vars, multi=True, spherical=True)
selector.search(multi_bw_min=[4])

In [ ]:
# Fit the GWR model to your data
mgwr_results = MGWR(coords, y, exp_vars, selector, sigma2_v1=True).fit()

# Print the results of the GWR model
print(mgwr_results)

mgwr_results.summary()

## MGWR with Binary Variables

In [ ]:
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW

In [ ]:
# Encoding categorical variables

# Room Type, All 0s = hotel room
listings_manhattan['rt_entire_home_apartment'] = np.where(listings_manhattan["room_type"]=='Entire home/apt', 1, 0)
listings_manhattan['rt_private_room'] = np.where(listings_manhattan["room_type"]=='Private room', 1, 0)
listings_manhattan['rt_shared_room'] = np.where(listings_manhattan["room_type"]=='Shared room', 1, 0)

# Cleaning up the price column
listings_manhattan['price'] = listings_manhattan['price'].str.replace('$', '')
listings_manhattan['price'] = listings_manhattan['price'].str.replace(',', '')
listings_manhattan['price'] = listings_manhattan['price'].astype(float)

# Logging the price variable
listings_manhattan['log_price'] = np.log(listings_manhattan['price'])

listings_manhattan = listings_manhattan[listings_manhattan['bedrooms'].notna()]
listings_manhattan = listings_manhattan[listings_manhattan['review_scores_rating'].notna()]
listings_manhattan = listings_manhattan[listings_manhattan['beds'].notna()]
listings_manhattan.isna().sum()

listings_manhattan = listings_manhattan[listings_manhattan['bedrooms'].notna()]
listings_manhattan = listings_manhattan[listings_manhattan['review_scores_rating'].notna()]
listings_manhattan = listings_manhattan[listings_manhattan['beds'].notna()]

In [ ]:
listings_manhattan_sample = listings_manhattan.copy()

In [ ]:
listings_manhattan_sample[['accommodates', 'bedrooms','review_scores_rating',
          'Central Park', 'Central Park Zoo', 'Empire State Building', 'Statue of Liberty', 
          'Rockeffeller Center', 'Chrysler Building', 'Times Square', 'MoMa', 'Charging Bull']]

In [ ]:
exp_vars = listings_manhattan_sample[['accommodates', 'bedrooms','review_scores_rating',
          'Central Park', 'Central Park Zoo', 'Empire State Building', 'Statue of Liberty', 
          'Rockeffeller Center', 'Chrysler Building', 'Times Square', 'MoMa', 'Charging Bull']].values
y = (listings_manhattan_sample['price'].values).reshape((-1,1))
coords = list(zip(listings_manhattan_sample.geometry.x,listings_manhattan_sample.geometry.y))

In [ ]:
# Standardizing
exp_vars = (exp_vars - exp_vars.mean(axis=0)) / exp_vars.std(axis=0)
y = (y - y.mean(axis=0)) / y.std(axis=0)

In [ ]:
try:
    selector = Sel_BW(coords, y, exp_vars, multi=True, spherical=True)
    selector.search(multi_bw_min=[15])
    result += 1
except:
    print('Doesnt work for ' + str(i))
    pass

In [ ]:
# Fit the GWR model to your data
mgwr_results = MGWR(coords, y, exp_vars, selector, fixed=False, sigma2_v1=True).fit()

# Print the results of the GWR model
print(mgwr_results)

In [ ]:
mgwr_results.summary()